This workflow is a working example designed to show how by using Autogen you can setup a team of three virtual analysts (agents) that can work together to: 

- extract and understand the structure of some synthetic marketing pipeline data
- aggregate and count marketing qualified leads, opportunities and deals for a sub-segment of the data
- group the data by MQL created date
- use that data to create a simple forecast for each metric
- comment on the results and suggest improvements and enhancements to the forecast
- create a visualisation of the forecast

Make sure you have autogen installed

In [ ]:
import autogen

Make sure you add in your api key here. This example has been tested with  'gpt-4-0125-preview' but it is possible to use almost any model. Results may vary however. 

In [2]:
config_list = [
    {
        'model': 'gpt-4-0125-preview',
        'api_key': 'yourapikeyhere'
    }
]



Change the seed number everytime you want to run the code without using a cached version of the previous run. 

In [3]:
gpt4_config = {
    "seed": 47,  # change the seed for different trials
    "temperature": 0,
    "config_list": config_list,
}
 

First we define what agents and tasks we want. 

For the first task in this list , make sure you add a data source where it states 'FILE PATH HERE'below. 

Example string for windows machine is here ->  C:\\Users\\username\\Documents\\autogen\\data\data_file_here.csv.
Obviously you need to change this to wherever your data is located.

I have included some synthetic data to experiment with in the repo. 

In [ ]:
data_analysis_tasks = [
    """The file I will share containes data for new business. FILE PATH HERE" Write a python script to inspect the file. Write another python script to create a seperate table that shows for each month in 2023 the number of MQLS (count of ids with 'Became an MQL Date'), the number of Opportunities (count of ids with Opportunity Created Date) and the number of closed won deals (count of ids with 'Closed Date' completed AND 'Stage Status' as 'Closed Won'). Group all data by MQL created date, but be sure to do this for 2023 data ONLY. Please make sure you save the new file.""",
   """write a python script that will create a forecast of MQLS, OPPS and Closed Won Deals in 2024, using only data from 2023 within the file we created as the initial input. Please use simple linear regression so we can see a baseline. Please make sure you save the 2023 and 2024 data as a csv file."""
]

sr_data_analyst_tasks = [
    """write a python script that will help the user print out the csv data saved, that covers 2023 and 2024. Wait until the user has executed this code and then comment on the results.""",

]
visualisation_tasks = ["""Write a Python script that will create a line chart for the 2023 and 2024 full forecast data."""]


Next we define the agents and comment on their roles. 

In [7]:
#for the coding tasks I have enabled the agents to be able to run code. This may need to be adjusted so the user proxy runs the code. TBD. If the user proxy runs the code, then we need any task that requires code to opearte as a groupchat where the assistant produes the code for the task, and the user runs the code. When the task is done the data is passed across.  


data_assistant= autogen.AssistantAgent(
    name="data_analyst",
    llm_config=gpt4_config,
    system_message="You are a helpful data analyst that is taking part in a conversation with a user. The user can execute any python code you produce. You are not expected to execute any code, nor do you have this ability. The user can execute code you produce and you will see the outputs of this code in the conversation once the user has executed. In this way you can create useful code that when passed to the user, can be used to generate outputs. You can then see these outputs once generated. When you believe you have achieved your task you MUST print TERMINATE ",
)

sr_data_assistant = autogen.AssistantAgent(
    name="sr_data_analyst",
    llm_config=gpt4_config,
    system_message="You are a helpful data analyst that is taking part in a conversation with a user. The user can execute any python code you produce. You are not expected to execute any code, nor do you have this ability. The user can execute code you produce and you will see the outputs of this code in the conversation ONLY once the user has executed. In this way you can create useful code that when passed to the user, can be used to generate outputs. You can then see these outputs once generated. You are a business expert who understands the strengths and weaknesses of forecasting methods for B2B SAAS. You can also write (but not execute) advanced python code as needed. You can analyse raw data in the chat, comment on what it may mean for the business, and also critique the methods used to produce the data, whilst suggesting alternatives. When you believe you have achieved your task you MUST print TERMINATE",
)


viz_assistant = autogen.AssistantAgent(
    name="viz_analyst",
    llm_config=gpt4_config,
    system_message= "When you believe you have achieved your task you MUST print TERMINATE"
)

user = autogen.UserProxyAgent(
    name="User",
    human_input_mode="ALWAYS",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={
        "last_n_messages": 1,
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
)


Then we define the order in which we want the tasks executed, who will execute the task, and importantly what type of information we want the agent to pass on to the next agent in the sequence. 

Running this cell will run the code. 

In [ ]:
chat_results = user.initiate_chats(
    [
        {
            "recipient": data_assistant,
            "message": data_analysis_tasks[0],
            "clear_history": False,
            "silent": False,
            "summary_method": "reflection_with_llm",
            "summary_prompt" :  "Please include a string summary (not a working link) of the paths to all csv files included in the conversation. If any tabular data is printed out in the conversation, please include those as well.",
        },
        {
            "recipient": data_assistant,
            "message": data_analysis_tasks[1],
            "summary_method": "reflection_with_llm",
            "summary_prompt" :  "Please include a string summary (not a working link) of the paths to all csv files included in the conversation. If any tabular data is printed out in the conversation, please include those as well.",
    
            "clear_history" : False,
        },
        {
            "recipient": sr_data_assistant,
            "message": sr_data_analyst_tasks[0],
            "summary_method": "reflection_with_llm",
            "summary_prompt" :  "Please include a string summary (not a working link) of the paths to all csv files included in the conversation. If any tabular data is printed out in the conversation, please include those as well.",
            "clear_history" : False,
        },
        {
            "recipient": viz_assistant,
            "message": visualisation_tasks[0],
            "summary_method": "reflection_with_llm",
            "summary_prompt" : "Please include a string summary (not a working link) of the paths to all csv files included in the conversation. If any tabular data is printed out in the conversation, please include those as well.",
            "clear_history" : False,
        },
        
    ]
)